In [1]:
import pandas as pd
import numpy as np


# 1:  READ DATA FILE & MAKE LIST OF EACH TYPE

In [2]:
path_slang = "datafile/abusive_slang.txt"
path_religious = "datafile/abusive_religious.txt"
path_personal_attack = "datafile/abusive_personal_attack.txt"
path_antifaminism = "datafile/abusive_antifaminism.txt"
path_political =  "datafile/abusive_political.txt"


def read_file(path):
    sent =[]
    sent.clear()
    with open(path, "r",encoding="utf-8") as f:
        for line in f:
            sent.append(line)
    return sent
slang_sent = read_file(path_slang)
religious_sent = read_file(path_religious)
personal_attack_sent = read_file(path_personal_attack)
political_sent = read_file(path_political)
antifaminism_sent = read_file(path_antifaminism)

In [3]:

for i in range(3):
    print(religious_sent[i])

﻿শয়তান অনেক বড় আলেম ও আবেদ ছিল

মাসুদ ছোট আলেম ছোট শয়তান

তুই যে ইহুদিদের পয়দা করা কুকুর



# 2. REMOVE ALL BAD CHARECTER

In [4]:
import re
def digit_remover(text):
    text = re.sub(r'[\d]','',text)
    text = re.sub(r'[?\!\t\‛\’\(\)\.\।]+','',text)
    text = re.sub(r'[/\-\,]',' ',text)
    text = text.rstrip("\n\r")
    return text


In [5]:
print(digit_remover("হত্যাকাণ্ড/খারাপ,অদৃশ্য-কাল্পনিক,ধর্মের/ধার্মিকদের,0"))

হত্যাকাণ্ড খারাপ অদৃশ্য কাল্পনিক ধর্মের ধার্মিকদের 


In [6]:
def remover(sent):
    for i in range(len(sent)):
        sent[i] = digit_remover(sent[i])
    return sent
slang_sent = remover(slang_sent)
religious_sent = remover(religious_sent)
personal_attack_sent = remover(personal_attack_sent)
political_sent = remover(political_sent)
antifaminism_sent = remover(antifaminism_sent)


In [7]:
for i in range(5):
    print(personal_attack_sent[i])


﻿তোদের কে বাশ দিব
টুপি টা খুল পান্জাবি টা ও খুল
মদ খা মদ খা মদ তর জন্য এখন আলাল চি চি চি মুসলীম এর জন্য হারাম জাতী সব জানে তুই দালালী করতেছত টাকার জন্য মাগীর পুলা তুই নাস্তীক 
ফকিন্নীর বাচ্চা যাদের খাবারগুলো খেয়েছিস দান খয়রাতগুলো খেয়েছিস তার জবাব তোকে দিতে হবে ফকিন্নীর বাচ্চা
ফকিন্নী শালা নেরাকুত্তা


# ADD COLUMN AND LABEL EACH TYPE OF DATA

In [8]:
slang_df = pd.DataFrame({'col':slang_sent})
slang_df['label'] = 1
religious_df = pd.DataFrame({'col':religious_sent})
religious_df['label'] = 2
personal_attack_df = pd.DataFrame({'col':personal_attack_sent})
personal_attack_df['label'] = 3
political_df = pd.DataFrame({'col':political_sent})
political_df['label'] = 4
antifaminism_df = pd.DataFrame({'col':antifaminism_sent})
antifaminism_df['label'] = 5

In [9]:
#attack_df.head()

# CONCATENATE EACH TYPE OF DATA AND SUFFLE THEM

In [10]:
frames = [slang_df, religious_df,personal_attack_df,political_df,antifaminism_df]
result = pd.concat(frames,ignore_index=True)
#suffle the data frame
result = result.sample(frac=1)

In [11]:
#result.head(5)

# SPLIT DATA INTO TRAIN AND TEST SET

In [12]:
from sklearn.model_selection import train_test_split
X = result['col']
y = result['label']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 1,test_size=.2)

In [13]:
#print(X_train.head())

In [14]:
result.shape

(2504, 2)

#  REPRESENTING TEXT AS NUMERICAL DATA

In [15]:
# import and instantiate tfidfVECTORIZER (with the default parameters)
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=lambda a: a.split())

In [16]:
# learn the 'vocabulary' of the training data (occurs in-place)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [17]:
#vect.get_feature_names()

# APPLY MULTINOMIAL NAIVE BAYES ALGORITHM

In [18]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=0.01)

In [19]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

Wall time: 12 ms


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [20]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [21]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.6487025948103793

# APPLY LINEAR SVC CLASSIFIER

In [22]:
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(X_train_dtm, y_train)
y_pred_class_svc = svc.predict(X_test_dtm)

In [23]:
metrics.accuracy_score(y_test, y_pred_class_svc)

0.6666666666666666

# APPLY LOGISTIC REGRESSION

In [24]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()
logit.fit(X_train_dtm, y_train)
y_pred_class_logit = logit.predict(X_test_dtm)

In [25]:
metrics.accuracy_score(y_test, y_pred_class_logit)

0.5868263473053892

# APPLY RANDOM FOREST CLASSIFIER

In [26]:
from sklearn.ensemble import RandomForestClassifier
randomf = RandomForestClassifier(n_estimators=1000,max_depth=50)
randomf.fit(X_train_dtm, y_train)
y_pred_class_randomf = randomf.predict(X_test_dtm)

In [27]:
metrics.accuracy_score(y_test, y_pred_class_logit)

0.5868263473053892